<a href="https://colab.research.google.com/github/iesous-kurios/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module2-sql-for-analysis/module_2_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DSPT3 Unit 3 Sprint 2 Module 2 Assignment


##Assignment

Reproduce (debugging as needed) the live lecture task of setting up and inserting the RPG data into a PostgreSQL database, and add the code you write to do so.

Then, set up a new table for the Titanic data (titanic.csv) - spend some time thinking about the schema to make sure it is appropriate for the columns. Enumerated types may be useful. Once it is set up, write a insert_titanic.py script that uses psycopg2 to connect to and upload the data from the csv, and add the file to your repo. Then start writing PostgreSQL queries to explore the data!


In [14]:
import psycopg2
from psycopg2.extras import execute_values
import json
import os
from dotenv import load_dotenv
import pandas as pd
load_dotenv() # looks inside the .env file for some env vars

# passes env var values to python var
DB_HOST = os.getenv("DB_HOST", default="OOPS")
DB_NAME = os.getenv("DB_NAME", default="OOPS")
DB_USER = os.getenv("DB_USER", default="OOPS")
DB_PASSWORD = os.getenv("DB_PASSWORD", default="OOPS")
connection = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST
)
cursor = connection.cursor()
#
#
#
print("------------")
insert_query = """
SELECT * FROM test_table;
"""
print(insert_query)
cursor.execute(insert_query)

#results = cursor.fetchone()
results = cursor.fetchall()
#print(results)
for row in results:
    print(row)



------------

SELECT * FROM test_table;

(1, 'A row name', None)
(2, 'Another row, with JSON', {'a': 1, 'b': ['dog', 'cat', 42], 'c': True})


Now that I know my .env file was loaded, and the test database was successfully connected to, and that a test SQL query was executed and printed, I can move on with the rest of the assignment

In [15]:
# use wget to pull rpg database into sqlite to then transfer into postgresql
!wget https://github.com/iesous-kurios/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true

# rename the above file to work with it more easily
!mv 'rpg_db.sqlite3?raw=true' rpg_db.sqlite3




--2020-02-03 06:11:59--  https://github.com/iesous-kurios/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/iesous-kurios/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2020-02-03 06:11:59--  https://github.com/iesous-kurios/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/iesous-kurios/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2020-02-03 06:11:59--  https://raw.githubusercontent.com/iesous-kurios/DS-Unit-3-Sprint-2-SQL-and-Databases/master/modul

In [0]:
import sqlite3

# established connection
sql_conn = sqlite3.connect('rpg_db.sqlite3')

# create a cursor
sql_curs = sql_conn.cursor()

# create python variable to hold list of all characters
characters = sql_curs.execute('SELECT * FROM charactercreator_character;').fetchall()

In [17]:
# take a peek into data to see if characters are in there

print(characters[10])

(11, 'Itaque ut commodi,', 0, 0, 10, 1, 1, 1, 1)


In [18]:
# get total number of characters

len(characters)

302

In [19]:
# Use sqlte to get schema for character table from rpg database
sql_curs.execute('PRAGMA table_info(charactercreator_character);').fetchall()

[(0, 'character_id', 'integer', 1, None, 1),
 (1, 'name', 'varchar(30)', 1, None, 0),
 (2, 'level', 'integer', 1, None, 0),
 (3, 'exp', 'integer', 1, None, 0),
 (4, 'hp', 'integer', 1, None, 0),
 (5, 'strength', 'integer', 1, None, 0),
 (6, 'intelligence', 'integer', 1, None, 0),
 (7, 'dexterity', 'integer', 1, None, 0),
 (8, 'wisdom', 'integer', 1, None, 0)]

In [0]:
# set up the types for moving data into postgresql

charactercreator_character = """ 
CREATE TABLE charactercreator_character(
  character_id SERIAL PRIMARY KEY,
  name VARCHAR(30),
  level INT,
  exp INT,
  hp INT,
  strength INT,
  intelligence INT,
  dexterity INT,
  wisdom INT
);
"""


In [0]:
# Switch to elephant sql and create an empty table in elephant sql called charactercreator_character

cursor.execute(charactercreator_character)

In [0]:
# verify that the created table is in elephant sql

show_tables = """
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog'
AND schemaname != 'information_schema';
"""

In [22]:
characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [23]:
# perform slice and remove id to insert data into postgress
str(characters[0][1:])

"('Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)"

In [0]:
# Insert all 302 characters from sqlite to elephantsql

for character in characters:
  insert_character = """
    INSERT INTO charactercreator_character
    (name, level, exp, hp, strength, intelligence, dexterity, wisdom)
    VALUES """ + str(character[1:]) + ';'
    #print(insert_character)
  cursor.execute(insert_character)

In [0]:
cursor.execute('SELECT * FROM charactercreator_character;')

In [25]:
cursor.fetchall()

[(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1),
 (2, 'Optio dolorem ex a', 0, 0, 10, 1, 1, 1, 1),
 (3, 'Minus c', 0, 0, 10, 1, 1, 1, 1),
 (4, 'Sit ut repr', 0, 0, 10, 1, 1, 1, 1),
 (5, 'At id recusandae expl', 0, 0, 10, 1, 1, 1, 1),
 (6, 'Non nobis et of', 0, 0, 10, 1, 1, 1, 1),
 (7, 'Perferendis', 0, 0, 10, 1, 1, 1, 1),
 (8, 'Accusantium amet quidem eve', 0, 0, 10, 1, 1, 1, 1),
 (9, 'Sed nostrum inventore error m', 0, 0, 10, 1, 1, 1, 1),
 (10, 'Harum repellendus omnis od', 0, 0, 10, 1, 1, 1, 1),
 (11, 'Itaque ut commodi,', 0, 0, 10, 1, 1, 1, 1),
 (12, 'Molestiae quis', 0, 0, 10, 1, 1, 1, 1),
 (13, 'Ali', 0, 0, 10, 1, 1, 1, 1),
 (14, 'Tempora quod optio possimus il', 0, 0, 10, 1, 1, 1, 1),
 (15, 'Sed itaque beatae pari', 0, 0, 10, 1, 1, 1, 1),
 (16, 'Quam dolor', 0, 0, 10, 1, 1, 1, 1),
 (17, 'Molestias expedita', 0, 0, 10, 1, 1, 1, 1),
 (18, 'Lauda', 0, 0, 10, 1, 1, 1, 1),
 (19, 'Incidunt sint perferen', 0, 0, 10, 1, 1, 1, 1),
 (20, 'Laboriosa', 0, 0, 10, 1, 1, 1, 1),
 (21,

In [0]:
# Save the changes (close and commit)
cursor.close()
connection.commit()

In [0]:
# Check that data in elephant sql matches whats in sqlite
postgress_curs =connection.cursor()
postgress_curs.execute('SELECT * FROM charactercreator_character;')
postgress_characters = postgress_curs.fetchall()

In [27]:
# compare the same character from each database to make sure they match
characters[5]

(6, 'Non nobis et of', 0, 0, 10, 1, 1, 1, 1)

In [28]:
postgress_characters[5]

(6, 'Non nobis et of', 0, 0, 10, 1, 1, 1, 1)

In [0]:
# Check that all the characters are the same
for character, postgress_character in zip(characters, postgress_characters):
  assert character == postgress_character